In [3]:
import sys
sys.path.append('..')
sys.path.append('/Users/jh7685/Documents/Projects/pysurfer')
import numpy as np
from pysurfer.mgz_helper import map_values_as_mgz
import os
from sfp_nsdsyn import *
import warnings
from itertools import product
warnings.filterwarnings("ignore", category=UserWarning)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Configurations

In [4]:
nsd_dir ='/Volumes/server/Projects/sfp_nsd/natural-scenes-dataset'
output_dir='/Volmes/server/Projects/sfp_nsd/derivatives'

In [5]:
subj='subj01'
hemi='lh'
stim_list = ['annulus', 'pinwheel', 'forward spiral', 'reverse spiral']
betas = os.path.join(nsd_dir, 'nsddata_betas', 'ppdata', f'{subj}', 'nativesurface', 'nsdsyntheticbetas_fithrf_GLMdenoise_RR', f'{hemi}.betas_nsdsynthetic.hdf5')
design_mat=os.path.join(nsd_dir, 'nsddata', 'experiments', 'nsdsynthetic', 'nsdsynthetic_expdesign.mat')
task_keys=['fixation_task','memory_task']
stim_info=os.path.join(nsd_dir, 'nsdsyn_stim_description.csv')
task_average=True
eccentricity_path= os.path.join(nsd_dir, 'nsddata', 'freesurfer', f'{subj}', 'label',  f'{hemi}.prfeccentricity.mgz')

# Load whole brain betas

In [6]:
betas_df = sfm.get_whole_brain_betas(betas_path=betas, design_mat_path=design_mat,
                          stim_info_path=stim_info,
                          task_keys=task_keys, task_average=False, eccentricity_path=eccentricity_path,
                          x_axis='voxel', y_axis='stim_idx', long_format=True)
betas_df = betas_df.query('names in @stim_list')

# Normalize betas by the frequency magnitude (?)

In [7]:
betas_df['normed_betas'] = bts.normalize_betas_by_frequency_magnitude(betas_df, betas='betas', freq_lvl='freq_lvl')

## NaN values???

In [107]:
tmp_df = betas_df.query('194587 < voxel < 194590')

bts.normalize_betas_by_frequency_magnitude(tmp_df, betas='betas', freq_lvl='freq_lvl')

0      0.363028
1      2.243133
2      0.791020
3      2.831621
4      0.401242
         ...   
379   -2.525187
380    0.310012
381    1.189318
382    0.000000
383    1.048404
Length: 384, dtype: float64

In [135]:
betas_df['normed_betas'] = bts.normalize_betas_by_frequency_magnitude(betas_df, betas='betas', freq_lvl='freq_lvl')
betas_df.query('voxel == 194589')

,stim_idx,image_idx,names,w_r,w_a,phase,class_idx,freq_lvl,voxel,task,betas,eccentricity,local_sf,normed_betas
43587936,0,104,pinwheel,0.0,6.0,0.000000,0,0,194589,fixation,0.353333,2.336213,6.000000,-0.071187
43587937,0,104,pinwheel,0.0,6.0,0.000000,0,0,194589,memory,2.076667,2.336213,6.000000,0.468191
43587938,1,105,pinwheel,0.0,6.0,1.570796,0,0,194589,fixation,1.293333,2.336213,6.000000,-0.102673
43587939,1,105,pinwheel,0.0,6.0,1.570796,0,0,194589,memory,3.410000,2.336213,6.000000,0.898051
43587940,2,106,pinwheel,0.0,6.0,3.141593,0,0,194589,fixation,0.130000,2.336213,6.000000,-0.267636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43588123,93,197,reverse spiral,91.0,-91.0,1.570796,23,5,194589,memory,-1.493333,2.336213,128.693434,NaN
43588124,94,198,reverse spiral,91.0,-91.0,3.141593,23,5,194589,fixation,0.183333,2.336213,128.693434,NaN
43588125,94,198,reverse spiral,91.0,-91.0,3.141593,23,5,194589,memory,0.703333,2.336213,128.693434,NaN
43588126,95,199,reverse spiral,91.0,-91.0,4.712389,23,5,194589,fixation,0.000000,2.336213,128.693434,NaN


In [168]:
new_df = betas_df.query('voxel == 194589')
a = bts.normalize_betas_by_frequency_magnitude(new_df, betas='betas', freq_lvl='freq_lvl')

new_df_2 = pd.merge(new_df[['voxel','betas']], a, on='voxel')
new_df_2['normed_betas'] = np.divide(new_df_2['betas'], new_df_2['length'])
new_df_2

,voxel,betas,length,normed_betas
0,194589,0.353333,0.591375,0.597477
1,194589,2.076667,0.591375,3.511589
2,194589,1.293333,0.591375,2.186993
3,194589,3.410000,0.591375,5.766221
4,194589,0.130000,0.591375,0.219827
...,...,...,...,...
187,194589,-1.493333,0.591375,-2.525187
188,194589,0.183333,0.591375,0.310012
189,194589,0.703333,0.591375,1.189318
190,194589,0.000000,0.591375,0.000000


In [171]:
betas_df = bts.normalize_betas_by_frequency_magnitude(betas_df, betas='betas', freq_lvl='freq_lvl')

In [117]:
tmp_df = betas_df.query('194587 < voxel < 194590')
tmp = tmp_df.groupby(['voxel', 'freq_lvl'])['betas'].mean().reset_index()
tmp = tmp.pivot('voxel', 'freq_lvl', 'betas')
length = np.linalg.norm(tmp, axis=1, keepdims=True)
length = np.concatenate((tmp.index.to_numpy().reshape(-1,1), length), axis=1)

# Get sigma_v for each voxel and map it in mgz

In [172]:
sigma_v = bts.get_sigma_v_for_whole_brain(betas_df, betas='normed_betas')

In [12]:
map_values_as_mgz(eccentricity_path, 1/sigma_v,         save_path=f'/Volumes/server/Projects/sfp_nsd/derivatives/sfp_maps/mgzs/nsdsyn/{hemi}.sub-{subj}_value-precision.mgz')

# Plot precision maps

In [22]:
import neuropythy as ny
import ipyvolume as ipv

%gui qt
ny.config['freesurfer_subject_paths'] = '/Volumes/server/Projects/sfp_nsd/natural-scenes-dataset/nsddata/freesurfer'

In [19]:
sub = ny.freesurfer_subject('/Volumes/server/Projects/sfp_nsd/natural-scenes-dataset/nsddata/freesurfer/subj01')
surf = sub.lh.surface('pial')
ny.cortex_plot(surf, color='thickness', cmap='hot', vmin=2, vmax=6)

RuntimeError: load failure: the requested object could not be loaded, probably because you do not have ipyvolume installed correctly